In [26]:
from lsst.pipe.tasks.imageDifference import ImageDifferenceTask

In [47]:
import glob
import os
import subprocess
import sys
import time
import math
import random
import numpy

from IPython.display import Image, display_png

import lsst.afw.display.ds9 as ds9
import lsst.afw.math as afwMath
import lsst.afw.geom as afwGeom
import lsst.afw.image as afwImage
import lsst.afw.table as afwTable

import lsst.ip.diffim as ipDiffim
import lsst.pex.config as pexConfig
import lsst.pipe.base as pipeBase
import lsst.daf.base as dafBase
import lsst.afw.geom as afwGeom
import lsst.afw.image as afwImage
import lsst.afw.math as afwMath
import lsst.afw.table as afwTable
import lsst.meas.astrom as measAstrom

from lsst.pipe.tasks.registerImage import RegisterTask
from lsst.meas.algorithms import PsfAttributes
#from lsst.meas.algorithms import SourceDetectionTask, SourceMeasurementTask, starSelectorRegistry, PsfAttributes, SingleGaussianPsf
from lsst.ip.diffim import ImagePsfMatchTask, DipoleMeasurementTask, DipoleAnalysis,SourceFlagChecker, KernelCandidateF, cast_KernelCandidateF, makeKernelBasisList
#    KernelCandidateQa, DiaCatalogSourceSelector, DiaCatalogSourceSelectorConfig
import lsst.ip.diffim.diffimTools as diffimTools




import lsst.daf.persistence as dafPersist
import lsst.pipe.tasks.imageDifference as ImageDifferencer
DATADIR="/renoir_data_02/jpreyes/lsst_winter2013/w13_diffim_v11/"





## Methods for ds9 Image Visualization

In [7]:
def displayFigure(path):
    i =  Image(path)
    display_png(i)
    
def displayImage(Image,frame=0,title="NoName",path="fig"):
    ds9.mtv(Image, frame=frame, title=title)
    ds9.ds9Cmd("zscale")
    ds9.ds9Cmd("zoom to fit")
    ds9.ds9Cmd("raise")
    ds9.ds9Cmd('saveimage '+path)
    print title
    displayFigure(path)

def showFrame(frame=0,title="NoTitle",path="fig"):
    ds9.ds9Cmd("zscale")
    ds9.ds9Cmd("zoom to fit")
    ds9.ds9Cmd("raise")
    ds9.ds9Cmd('saveimage '+path)
    print title
    displayFigure(path)
    
def displayImageWithSources(Image,frame=0,title="NoName",path="fig", sources=[]):
    ds9.mtv(Image, frame=frame, title=title)
    ds9.ds9Cmd("zscale")
    ds9.ds9Cmd("zoom to fit")
    ds9.ds9Cmd("raise")
    
    
    for source in sources:
        ds9.dot("+", source.getX()-Image.getX0()  , source.getY()-Image.getY0() , frame=frame, size = 25, ctype = ds9.RED)

    
    ds9.ds9Cmd('saveimage '+path)
    print title
    displayFigure(path)


## Methods for pre-processing the input data

In [3]:
if(os.path.exists(DATADIR)==False):
    os.makedirs(DATADIR)
if(os.path.exists(DATADIR+"/input")==False):
    os.makedirs(DATADIR+"/input")
temp_file=open(DATADIR+"/input/_mapper",'w')
temp_file.write('lsst.obs.cfht.MegacamMapper')
temp_file.close()


In [106]:
ITask = Ingester.IngestTask
IConfig = ITask.ConfigClass()
IParser = ITask.ArgumentParser("ingest")
Files = glob.glob("/renoir_data_02/jpreyes/lsst_simulated_data/sparse_diffim_data/raw/v6866601-fi/E000/R22/S00/*fits.gz")
SFiles = ' '.join(Files)
print SFiles
args = IParser.parse_args(IConfig, (DATADIR+'/input '+SFiles+' --mode link').split()) 
task = ITask(config=args.config)
task.run(args)

NameError: name 'Ingester' is not defined

In [ ]:
PTask = Processer.ProcessCcdTask

PTask.parseAndRun(args=((DATADIR+"/input --output "+DATADIR+"/output --id ccd=14 --config isr.doBias=False isr.doFlat=False isr.doFringe=False").split()))

# ImageDifferenceTask

Initialization of classes, config and other important variables. Most "self" references in the ImageDifferenceTask should be replaced by IDifTask. self.config references can be replaced by IDConfig

In [108]:
config = ipDiffim.ImagePsfMatchTask.ConfigClass()
config.kernel.name = "AL"
subconfig = config.kernel.active
defFwhm         = None
scienceFwhmPix = defFwhm
templateFwhmPix = defFwhm
psfmatch = ipDiffim.ImagePsfMatchTask(config)

In [109]:
IDTask = ImageDifferencer.ImageDifferenceTask
IDConfig = IDTask.ConfigClass()
IDifTask = IDTask()
IDifTask.__init__()

FwhmPerSigma = 2 * math.sqrt(2 * math.log(2))
IqrToSigma = 0.741

Intialization of outputs and intermediate products

In [110]:
subtractedExposure = None
subtractRes = None
selectSources = None
kernelSources = None
controlSources = None
diaSources = None


Butler initialization. Here we have to indicate the image which is going to be used for the processing.

In [111]:
butler = dafPersist.Butler(DATADIR)
dataid = {'visit': 6866602, 'filter':'i' , 'raft':'2,2', 'sensor':'1,1'}
templateid= {'visit': 88868666, 'filter':'i' , 'raft':'2,2', 'sensor':'1,1'}

In [112]:
exposure = butler.get("calexp", dataid)

expBits = butler.get("ccdExposureId_bits", dataid)
expId = long(butler.get("ccdExposureId", dataid))
idFactory = afwTable.IdFactory.makeSource(expId, 64 - expBits)


exposureBg = butler.get("calexpBackground", dataid)

sciencePsf = exposure.getPsf()

In [113]:
expBits = butler.get("ccdExposureId_bits", dataid)
expId = long(butler.get("ccdExposureId", dataid))
idFactory = afwTable.IdFactory.makeSource(expId, 64 - expBits)

        # Retrieve the science image we wish to analyze
exposure = butler.get("calexp", dataid, immediate=True)
if IDConfig.doAddCalexpBackground:
            mi = exposure.getMaskedImage()
            mi += butler.get("calexpBackground", dataid).getImage()
if not exposure.hasPsf():
       raise pipeBase.TaskError("Exposure has no psf")
sciencePsf = exposure.getPsf()


There are two methods for obtaining the templateExposure. Those method rely on coadd tract info already generated (pre-processing) or prepared. As a result, these two lines have to be change in order to reflect a real templateExposure for the given image.

In [114]:
templateExposure = butler.get("calexp", templateid)
templateSources =  butler.get("src", templateid)

Compute the sigma of Science Image and Template Image PSF before convolution

In [115]:
ctr = afwGeom.Box2D(exposure.getBBox(afwImage.PARENT)).getCenter()
psfAttr = PsfAttributes(sciencePsf, afwGeom.Point2I(ctr))
scienceSigmaOrig = psfAttr.computeGaussianWidth(psfAttr.ADAPTIVE_MOMENT)

ctr = afwGeom.Box2D(templateExposure.getBBox(afwImage.PARENT)).getCenter()
psfAttr = PsfAttributes(templateExposure.getPsf(), afwGeom.Point2I(ctr))
templateSigma = psfAttr.computeGaussianWidth(psfAttr.ADAPTIVE_MOMENT)

Convolve the science Exposure with its PSF (as mentioned on the Alard-Luptor article), or simply put the hard-value on it.

In [116]:
if IDConfig.doPreConvolve:
     print "doing Pre-convolve"
     convControl = afwMath.ConvolutionControl()
     # cannot convolve in place, so make a new MI to receive convolved image
     srcMI = exposure.getMaskedImage()
     destMI = srcMI.Factory(srcMI.getDimensions())
     srcPsf = sciencePsf
     if IDConfig.useGaussianForPreConvolution:
         # convolve with a simplified PSF model: a double Gaussian
         kWidth, kHeight = sciencePsf.getLocalKernel().getDimensions()
         preConvPsf = SingleGaussianPsf(kWidth, kHeight, scienceSigmaOrig)
     else:
         print "no Pre-Convolve is being done"
         # convolve with science exposure's PSF model
         preConvPsf = psf
         afwMath.convolve(destMI, srcMI, preConvPsf.getLocalKernel(), convControl)
         exposure.setMaskedImage(destMI)
         scienceSigmaPost = scienceSigmaOrig * math.sqrt(2)
else:
     scienceSigmaPost = scienceSigmaOrig


Search for the sources on the Image

In [117]:
scienceSigmaPost = scienceSigmaOrig
print(dir(IDTask))
selectSources = psfmatch.getSelectSources(
                        exposure,
                        sigma = scienceSigmaPost,
                        doSmooth = not False,
                        idFactory = None)

['ConfigClass', 'RunnerClass', '_DefaultName', '__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_computeFullName', '_getConfigName', '_getMetadataName', '_makeArgumentParser', 'applyOverrides', 'canMultiprocess', 'display', 'emptyMetadata', 'fitAstrometry', 'getAllSchemaCatalogs', 'getFullMetadata', 'getFullName', 'getName', 'getSchemaCatalogs', 'getTaskDict', 'makeField', 'makeSubtask', 'parseAndRun', 'run', 'runDebug', 'timer', 'writeConfig', 'writeMetadata', 'writeSchemas']


Number of Basis functions (Alard-Lupton Method)

In [118]:
nparam = len(makeKernelBasisList(IDifTask.subtract.config.kernel.active,
                                                 referenceFwhmPix=scienceSigmaPost * FwhmPerSigma,
                                                 targetFwhmPix=templateSigma * FwhmPerSigma))


Modifying schemas... (not working sometimes)

In [119]:
IDifTask.config.doAddMetrics = False
if IDifTask.config.doAddMetrics:
        print "Modify the schema of all Sources"
        kcQa = KernelCandidateQa(nparam)
        selectSources = kcQa.addToSchema(selectSources)


Some operations with the sources

In [120]:
#astrometer = measAstrom.AstrometryTask()
astrometer = measAstrom.AstrometryTask()

In [121]:
astrometer = pexConfig.ConfigurableField(
        target = measAstrom.AstrometryTask,
        doc = "astrometry task; used to match sources to reference objects, but not to fit a WCS",
    )

#print IDifTask.config.doUseRegister
#print astrometer

IDifTask.sourceSelector = IDifTask.config.sourceSelector.apply()

#astrometer = IDifTask.makeSubtask("astrometer")

astroConfig = measAstrom.AstrometryConfig()
astrometer=measAstrom.AstrometryTask(astroConfig)


In [122]:
astromRet = astrometer.loadAndMatch(exposure=exposure, sourceCat=selectSources)
matches = astromRet.matches
kernelSources = self.sourceSelector.selectSources(exposure, selectSources, matches=matches)
random.shuffle(kernelSources, random.random)
controlSources = kernelSources[::self.config.controlStepSize]
kernelSources = [k for i,k in enumerate(kernelSources) if i % self.config.controlStepSize]

RuntimeError: Unable to find andConfig.py in astrometry_net_data directory /renoir_data_00/fouchez/lsst/DM/stacks/v11.0/Linux64/astrometry_net_data/10.0+41

<b>Extended DiaSources Selection:</b> Not working! This part depends on "stargal" flag

In [87]:
if IDifTask.config.doSelectDcrCatalog:
    print "Selecting from DCR Catalog"
    redSelector  = DiaCatalogSourceSelector(
        DiaCatalogSourceSelectorConfig(grMin=IDifTask.sourceSelector.config.grMax, grMax=99.999))
    redSources   = redSelector.selectSources(exposure, selectSources, matches=matches)
    controlSources.extend(redSources)
    
    blueSelector = DiaCatalogSourceSelector(
        DiaCatalogSourceSelectorConfig(grMin=-99.999, grMax=IDifTask.sourceSelector.config.grMin))
    blueSources  = blueSelector.selectSources(exposure, selectSources, matches=matches)
    controlSources.extend(blueSources)      


<b>Select Variable Catalog</b>: Not working! This part depends on "stargal" flag

In [88]:
if IDifTask.config.doSelectVariableCatalog:
    print "Selecting from Variable Catalog"
    varSelector = DiaCatalogSourceSelector(DiaCatalogSourceSelectorConfig(includeVariable=True))
    varSources  = varSelector.selectSources(exposure, selectSources, matches=matches)
    controlSources.extend(varSources)


Fit the relative astrometry between the two images. Template is warped to match science exposure

In [40]:
wcsResults = IDifTask.fitAstrometry(templateSources, templateExposure, selectSources)
warpedExp = IDifTask.register.warpExposure(templateExposure, wcsResults.wcs,
                                            exposure.getWcs(), exposure.getBBox(afwImage.PARENT))
templateExposure = warpedExp

This creates a lot of important metrics based on the matching method

In [42]:
IDifTask.config.doDebugRegister = True
if IDifTask.config.doDebugRegister:
                    print "registering debugging outputs for residuals et al"
                    # Grab matches to reference catalog
                    srcToMatch = {x.second.getId() : x.first for x in matches}

                    refCoordKey = wcsResults.matches[0].first.getTable().getCoordKey()
                    inCentroidKey = wcsResults.matches[0].second.getTable().getCentroidKey()
                    sids      = [m.first.getId() for m in wcsResults.matches]
                    positions = [m.first.get(refCoordKey) for m in wcsResults.matches]
                    residuals = [m.first.get(refCoordKey).getOffsetFrom(wcsResults.wcs.pixelToSky(
                                m.second.get(inCentroidKey))) for m in wcsResults.matches]
                    allresids = dict(zip(sids, zip(positions, residuals)))

                    cresiduals = [m.first.get(refCoordKey).getTangentPlaneOffset(
                            wcsResults.wcs.pixelToSky(
                                m.second.get(inCentroidKey))) for m in wcsResults.matches]
                    colors    = numpy.array([-2.5*numpy.log10(srcToMatch[x].get("g"))
                                              + 2.5*numpy.log10(srcToMatch[x].get("r"))
                                              for x in sids if x in srcToMatch.keys()])
                    dlong     = numpy.array([r[0].asArcseconds() for s,r in zip(sids, cresiduals)
                                             if s in srcToMatch.keys()])
                    dlat      = numpy.array([r[1].asArcseconds() for s,r in zip(sids, cresiduals)
                                             if s in srcToMatch.keys()])
                    idx1      = numpy.where(colors<IDifTask.sourceSelector.config.grMin)
                    idx2      = numpy.where((colors>=IDifTask.sourceSelector.config.grMin)&
                                            (colors<=IDifTask.sourceSelector.config.grMax))
                    idx3      = numpy.where(colors>IDifTask.sourceSelector.config.grMax)
                    print idx1, idx2, idx3
'''
                    rms1Long  = IqrToSigma*(numpy.percentile(dlong[idx1],75)-numpy.percentile(dlong[idx1],25))
                    rms1Lat   = IqrToSigma*(numpy.percentile(dlat[idx1],75)-numpy.percentile(dlat[idx1],25))
                    rms2Long  = IqrToSigma*(numpy.percentile(dlong[idx2],75)-numpy.percentile(dlong[idx2],25))
                    rms2Lat   = IqrToSigma*(numpy.percentile(dlat[idx2],75)-numpy.percentile(dlat[idx2],25))
                    rms3Long  = IqrToSigma*(numpy.percentile(dlong[idx3],75)-numpy.percentile(dlong[idx3],25))
                    rms3Lat   = IqrToSigma*(numpy.percentile(dlat[idx3],75)-numpy.percentile(dlat[idx3],25))
                    IDifTask.log.info("Blue star offsets'': %.3f %.3f, %.3f %.3f"  % (numpy.median(dlong[idx1]),
                                                                                  rms1Long,
                                                                                  numpy.median(dlat[idx1]),
                                                                                  rms1Lat))
                    IDifTask.log.info("Green star offsets'': %.3f %.3f, %.3f %.3f"  % (numpy.median(dlong[idx2]),
                                                                                   rms2Long,
                                                                                   numpy.median(dlat[idx2]),
                                                                                   rms2Lat))
                    IDifTask.log.info("Red star offsets'': %.3f %.3f, %.3f %.3f"  % (numpy.median(dlong[idx3]),
                                                                                 rms3Long,
                                                                                 numpy.median(dlat[idx3]),
                                                                                 rms3Lat))

                    IDifTask.metadata.add("RegisterBlueLongOffsetMedian", numpy.median(dlong[idx1]))
                    IDifTask.metadata.add("RegisterGreenLongOffsetMedian", numpy.median(dlong[idx2]))
                    IDifTask.metadata.add("RegisterRedLongOffsetMedian", numpy.median(dlong[idx3]))
                    IDifTask.metadata.add("RegisterBlueLongOffsetStd", rms1Long)
                    IDifTask.metadata.add("RegisterGreenLongOffsetStd", rms2Long)
                    IDifTask.metadata.add("RegisterRedLongOffsetStd", rms3Long)

                    IDifTask.metadata.add("RegisterBlueLatOffsetMedian", numpy.median(dlat[idx1]))
                    IDifTask.metadata.add("RegisterGreenLatOffsetMedian", numpy.median(dlat[idx2]))
                    IDifTask.metadata.add("RegisterRedLatOffsetMedian", numpy.median(dlat[idx3]))
'''

registering debugging outputs for residuals et al
(array([], dtype=int64),) (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73]),) (array([], dtype=int64),)


'\n                    rms1Long  = IqrToSigma*(numpy.percentile(dlong[idx1],75)-numpy.percentile(dlong[idx1],25))\n                    rms1Lat   = IqrToSigma*(numpy.percentile(dlat[idx1],75)-numpy.percentile(dlat[idx1],25))\n                    rms2Long  = IqrToSigma*(numpy.percentile(dlong[idx2],75)-numpy.percentile(dlong[idx2],25))\n                    rms2Lat   = IqrToSigma*(numpy.percentile(dlat[idx2],75)-numpy.percentile(dlat[idx2],25))\n                    rms3Long  = IqrToSigma*(numpy.percentile(dlong[idx3],75)-numpy.percentile(dlong[idx3],25))\n                    rms3Lat   = IqrToSigma*(numpy.percentile(dlat[idx3],75)-numpy.percentile(dlat[idx3],25))\n                    IDifTask.log.info("Blue star offsets\'\': %.3f %.3f, %.3f %.3f"  % (numpy.median(dlong[idx1]),\n                                                                                  rms1Long,\n                                                                                  numpy.median(dlat[idx1]),\n             

Subtraction is finally done!

In [43]:
subtractRes = IDifTask.subtract.subtractExposures(
                templateExposure=templateExposure,
                scienceExposure=exposure,
                scienceFwhmPix=scienceSigmaPost * FwhmPerSigma,
                templateFwhmPix=templateSigma * FwhmPerSigma,
                candidateList=kernelSources,
                convolveTemplate=IDConfig.convolveTemplate,
                doWarping=not IDConfig.doUseRegister
            )
subtractedExposure = subtractRes.subtractedExposure

PSF is assigned appropiately if necessary

In [44]:
if not subtractedExposure.hasPsf():
    if IDConfig.convolveTemplate:
       subtractedExposure.setPsf(exposure.getPsf())
else:
    if templateExposure is None:
            #templateExposure, templateSources = self.getTemplate(exposure, sensorRef)
       subtractedExposure.setPsf(templateExposure.getPsf())



Erase existing detection mask planes (not working for some unknown reason)

In [45]:
#DifTaskmask  = subtractedExposure.getMaskedImage().getMask()
#mask &= ~(mask.getPlaneBitMask("DETECTED") | mask.getPlaneBitMask("DETECTED_NEGATIVE"))

SourceCatalog construction based on old results

In [46]:
table = afwTable.SourceTable.make(IDifTask.schema, idFactory)
table.setMetadata(IDifTask.algMetadata)
results = IDifTask.detection.makeSourceCatalog(
                table=table,
                exposure=subtractedExposure,
                doSmooth=not IDConfig.doPreConvolve
                )

Do Merge

In [47]:
if IDifTask.config.doMerge:
    fpSet = results.fpSets.positive
    fpSet.merge(results.fpSets.negative, IDConfig.growFootprint,
                   IDConfig.growFootprint, False)
    diaSources = afwTable.SourceCatalog(table)
    fpSet.makeSources(diaSources)
else:
    diaSources = results.sources


This make the measurement of the Dia sources that are dipoles

In [48]:
if IDifTask.config.doMeasurement:
    print "Doing measurement of dipoles"
    if len(diaSources) < IDifTask.config.maxDiaSourcesToMeasure:
         IDifTask.dipoleMeasurement.run(subtractedExposure, diaSources)
    else:
         IDifTask.measurement.run(subtractedExposure, diaSources)


Doing measurement of dipoles


Match the Dia Sources with the Calexp Sources if possible

In [49]:

if IDifTask.config.doMatchSources:
    print "Matching sources"
    if butler.datasetExists("src", dataid):
                    # Create key,val pair where key=diaSourceId and val=sourceId
                    matchRadAsec = IDifTask.config.diaSourceMatchRadius
                    matchRadPixel = matchRadAsec / exposure.getWcs().pixelScale().asArcseconds()

                    srcMatches = afwTable.matchXy(butler.get("src", dataid), diaSources, matchRadPixel, True)
                    srcMatchDict = dict([(srcMatch.second.getId(), srcMatch.first.getId()) for
                                       srcMatch in srcMatches])

    else:
                    srcMatchDict = {}


Matching sources


Assign the pairs for sources and reference Id

In [50]:
astrometer = measAstrom.Astrometry(measAstrom.MeasAstromConfig(catalogMatchDist=matchRadAsec))
astromRet = astrometer.useKnownWcs(diaSources, exposure=exposure)
refMatches = astromRet.matches
if refMatches is None:
    refMatchDict = {}
else:
    refMatchDict = dict([(refMatch.second.getId(), refMatch.first.getId()) for \
                                             refMatch in refMatches])


Assign the DIA sources IDs

In [51]:
for diaSource in diaSources:
  sid = diaSource.getId()
  if srcMatchDict.has_key(sid):
        diaSource.set("srcMatchId", srcMatchDict[sid])
  if refMatchDict.has_key(sid):
        diaSource.set("refMatchId", refMatchDict[sid])


This method assign metrics if they exist. It is better to let it in "NO", as there are some problems with C++ classes compatibility

In [52]:
IDifTask.config.doAddMetrics = False
if IDifTask.config.doAddMetrics and IDifTask.config.doSelectSources:
     print "Adding metrics and sources"
     kernelCandList = []
     for cell in subtractRes.kernelCellSet.getCellList():
           for cand in cell.begin(False): # include bad candidates
                kernelCandList.append(cast_KernelCandidateF(cand))

                # Get basis list to build control sample kernels
     basisList = afwMath.cast_LinearCombinationKernel(
                    kernelCandList[0].getKernel(KernelCandidateF.ORIG)).getKernelList()

     controlCandList = \
            diffimTools.sourceTableToCandidateList(controlSources,
                                                           subtractRes.warpedExposure, exposure,
                                                           IDifTask.config.subtract.kernel.active,
                                                           IDifTask.config.subtract.kernel.active.detectionConfig,
                                                           IDifTask.log, doBuild=True, basisList=basisList)

     kcQa.apply(kernelCandList, subtractRes.psfMatchingKernel, subtractRes.backgroundModel,
                                dof=nparam)
     kcQa.apply(controlCandList, subtractRes.psfMatchingKernel, subtractRes.backgroundModel)
            
     if IDifTask.config.doDetection:
          kcQa.aggregate(selectSources, IDifTask.metadata, allresids, diaSources)
     else:
          kcQa.aggregate(selectSources, IDifTask.metadata, allresids)



Match with the calexp sources if possible

In [53]:
if butler.datasetExists("src", dataid):
    matchRadAsec = IDConfig.diaSourceMatchRadius
    matchRadPixel = matchRadAsec / exposure.getWcs().pixelScale().asArcseconds()
    srcMatches = afwTable.matchXy(butler.get("src", dataid), diaSources, matchRadPixel, True)
    srcMatchDict = dict([(srcMatch.second.getId(), srcMatch.first.getId()) for
                                             srcMatch in srcMatches])


Display the final Image

In [54]:
ds9.setMaskTransparency(50)
displayImage(subtractedExposure.getMaskedImage(),frame=0,title="SubtractedResult",path="subResult")

SubtractedResult
